# **PROYECTO UNIDAD 3**
- Integrantes 
 - Noelia Ancundia 
 - Wilmer Shagñay

#**MODELO ESTRELLA**

##**Librerias**

---



In [1]:
import sqlite3
import numpy
import glob
import pandas as pd
import sqlalchemy

##Crear dimensiones y base de datos 

---



Función de crear dimensiones

In [2]:
def crear_dimension(query):
    conexion=sqlite3.connect("DW_estrella.db")
    try:
        conexion.execute(query)
        print("se creo la tabla")                        
    except sqlite3.OperationalError:
        print("La tabla ingresada ya existe")                    
    conexion.close()

Crear la dimensión custumer

In [3]:
customer_query = '''CREATE TABLE dim_Customer (
                    Id_custumer  VARCHAR (8000) PRIMARY KEY,
                    cus_Address      VARCHAR (8000),
                    cus_City         VARCHAR (8000),
                    cus_Country      VARCHAR (8000),
                    cus_Phone        VARCHAR (8000)
                );'''

crear_dimension(customer_query)

se creo la tabla


Crear la dimensión employees

In [4]:
employees_query = '''CREATE TABLE dim_Employee (
                      Id_employee     INTEGER        PRIMARY KEY,
                      emp_LastName    VARCHAR (8000),
                      emp_FirstName   VARCHAR (8000),
                      emp_Title       VARCHAR (8000),
                      emp_Address     VARCHAR (8000),
                      emp_City        VARCHAR (8000),
                      emp_Country     VARCHAR (8000)
                  );
                  '''

crear_dimension(employees_query)

se creo la tabla


Crear la dimensión producto

In [5]:
producto_query = '''
                  CREATE TABLE dim_Product (
                      Id_produc       INTEGER PRIMARY KEY,
                      ProductName     VARCHAR (8000),
                      CategoryName VARCHAR (8000)
                  );'''
crear_dimension(producto_query)

se creo la tabla


Crear la dimensión supplier

In [6]:
supplier_query = '''CREATE TABLE dim_Supplier (
                    Id_supplier      INTEGER PRIMARY KEY NOT NULL,
                    sup_CompanyName  VARCHAR (8000),
                    sup_Address      VARCHAR (8000),
                    sup_City         VARCHAR (8000),
                    sup_Country      VARCHAR (8000)
                );'''
crear_dimension(supplier_query)

se creo la tabla


Crear la dimensión Tiempo

In [7]:
time_query = '''CREATE TABLE dim_time (
                Id_time    INTEGER       PRIMARY KEY
                                        NOT NULL,
                Date       DATE          NOT NULL,
                Day        INTEGER       NOT NULL,
                Month      INTEGER       NOT NULL,
                Year       INTEGER       NOT NULL,
                Quarter    INTEGER       NOT NULL,
                Week       INTEGER       NOT NULL,
                WeekDay    INTEGER       NOT NULL,
                Name_Day   NVARCHAR (10) NOT NULL,
                Name_Month NVARCHAR (15) NOT NULL
                );
                '''
crear_dimension(time_query)

se creo la tabla


Crear la Tabla de hechos 

In [8]:
hecho_query = '''CREATE TABLE Fact_sales_order (
                  Id_hechos     INTEGER         PRIMARY KEY AUTOINCREMENT
                                                NOT NULL,
                  Id_custumer   VARCHAR (8000)  REFERENCES dim_Customer (Id_custumer),
                  Id_employee   INTEGER         REFERENCES dim_Employee (Id_employee) 
                                                NOT NULL,
                  Id_time       INTEGER         REFERENCES dim_time (Id_time) 
                                                NOT NULL,
                  Id_supplier   INTEGER         REFERENCES dim_Supplier (Id_supplier) 
                                                NOT NULL,
                  Id_produc     INTEGER         REFERENCES dim_Product (Id_produc) 
                                                NOT NULL,
                  Quantity      INTEGER         NOT NULL,
                  UnitPrice     DECIMAL         NOT NULL,
                  Total         NUMERIC (10, 2) NOT NULL
              );'''

crear_dimension(hecho_query)

se creo la tabla


# **Proceso ETL**

---





## **Conexión a la base de datos**


Conexión a la base de datos Northwind_large

In [9]:
path = "sqlite:///Northwind_large.sqlite"
motorDB = sqlalchemy.create_engine(path)
conectarDB = motorDB.connect()


##**EXTRACCIÓN**

---



Función del proceso de extracción

In [10]:
def extraer_datos_sql(conectarDB,query):
    result = conectarDB.execute(query)
    df = pd.DataFrame(result.fetchall())
    df.columns = result.keys()
    return df

Extracción de datos para la dimension custumers 

In [11]:
query = '''SELECT Id AS Id_custumer,
                  Address AS cus_Address,
                  City AS cus_City,
                  Country AS cus_Country,
                  Phone AS cus_Phone
              FROM Customer;'''
df_custumer = extraer_datos_sql(conectarDB,query)
df_custumer.head()


,Id_custumer,cus_Address,cus_City,cus_Country,cus_Phone
0,ALFKI,Obere Str. 57,Berlin,Germany,030-0074321
1,ANATR,Avda. de la Constitución 2222,México D.F.,Mexico,(5) 555-4729
2,ANTON,Mataderos 2312,México D.F.,Mexico,(5) 555-3932
3,AROUT,120 Hanover Sq.,London,UK,(171) 555-7788
4,BERGS,Berguvsvägen 8,Luleå,Sweden,0921-12 34 65


Extracción de datos para la dimension employees 

In [12]:
query = '''SELECT LastName AS emp_LastName,
                  FirstName AS emp_FirstName,
                  Title AS emp_Title,
                  Address AS emp_Address,
                  City AS emp_City,
                  Country AS emp_Country
              FROM Employee;'''
df_employees = extraer_datos_sql(conectarDB,query)
df_employees.head()

,emp_LastName,emp_FirstName,emp_Title,emp_Address,emp_City,emp_Country
0,Davolio,Nancy,Sales Representative,507 - 20th Ave. E. Apt. 2A,Seattle,USA
1,Fuller,Andrew,"Vice President, Sales",908 W. Capital Way,Tacoma,USA
2,Leverling,Janet,Sales Representative,722 Moss Bay Blvd.,Kirkland,USA
3,Peacock,Margaret,Sales Representative,4110 Old Redmond Rd.,Redmond,USA
4,Buchanan,Steven,Sales Manager,14 Garrett Hill,London,UK


Extracción de datos para la dimension product 

In [13]:
query = '''SELECT Product.ProductName,
                  Category.CategoryName
              FROM Product
                  INNER JOIN
                  Category ON Category.Id = Product.CategoryId;'''
df_product = extraer_datos_sql(conectarDB,query)
df_product.head()

,ProductName,CategoryName
0,Chai,Beverages
1,Chang,Beverages
2,Aniseed Syrup,Condiments
3,Chef Anton's Cajun Seasoning,Condiments
4,Chef Anton's Gumbo Mix,Condiments


Extracción de datos para la dimension Supplier 

In [14]:
query = '''SELECT CompanyName AS sup_CompanyName,
                  Address AS sup_Address,
                  City AS sup_City,
                  Country AS sup_Country
              FROM Supplier;'''
df_supplier = extraer_datos_sql(conectarDB,query)
df_supplier.head()

,sup_CompanyName,sup_Address,sup_City,sup_Country
0,Exotic Liquids,49 Gilbert St.,London,UK
1,New Orleans Cajun Delights,P.O. Box 78934,New Orleans,USA
2,Grandma Kelly's Homestead,707 Oxford Rd.,Ann Arbor,USA
3,Tokyo Traders,9-8 Sekimai Musashino-shi,Tokyo,Japan
4,Cooperativa de Quesos 'Las Cabras',Calle del Rosal 4,Oviedo,Spain


Extracción de datos para la dimension Time 

In [15]:
date = pd.date_range('2006-02-19','2012-07-30',freq='D').to_series()
dates = pd.to_datetime(date) 
day = dates.dt.day
month = dates.dt.month
year = dates.dt.year
quarter = dates.dt.quarter
week = dates.dt.week
weekday = dates.dt.weekday
day_name = dates.dt.day_name()
month_name = dates.dt.month_name()
    
df_date = pd.DataFrame({'Date':list(dates),'Day':list(day),'Month':list(month),'Year':list(year),
                        'Quarter':list(quarter),'Week':list(week),'Weekday':list(weekday),
                        'Name_Day':list(day_name),'Name_Month':list(month_name)})
df_date.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  import sys


,Date,Day,Month,Year,Quarter,Week,Weekday,Name_Day,Name_Month
0,2006-02-19,19,2,2006,1,7,6,Sunday,February
1,2006-02-20,20,2,2006,1,8,0,Monday,February
2,2006-02-21,21,2,2006,1,8,1,Tuesday,February
3,2006-02-22,22,2,2006,1,8,2,Wednesday,February
4,2006-02-23,23,2,2006,1,8,3,Thursday,February


Extracción de datos para la tabla de hechos

In [16]:
query = '''SELECT C.Id AS Id_custumer,
                  E.Id AS Id_employee,
                  D.Id AS Id_time,
                  S.Id AS Id_supplier,
                  P.Id AS Id_produc,
                  O.Quantity,
                  O.UnitPrice,
                  (O.Quantity * O.UnitPrice) AS Total
              FROM Orders D
                  INNER JOIN
                  OrderDetail O ON O.OrderId = D.Id
                  INNER JOIN
                  Customer C ON C.Id = D.CustomerId
                  INNER JOIN
                  Employee E ON E.Id = D.EmployeeId
                  INNER JOIN
                  Product P ON P.Id = O.ProductId
                  INNER JOIN
                  Supplier S ON S.Id = P.SupplierId;'''
df_hechos = extraer_datos_sql(conectarDB,query)
df_hechos.head()

,Id_custumer,Id_employee,Id_time,Id_supplier,Id_produc,Quantity,UnitPrice,Total
0,VINET,5,10248,5,11,12,14.0,168.0
1,VINET,5,10248,20,42,10,9.8,98.0
2,VINET,5,10248,14,72,5,34.8,174.0
3,TOMSP,6,10249,6,14,9,18.6,167.4
4,TOMSP,6,10249,24,51,40,42.4,1696.0


# **TRANSFORMACIÓN**

---



Transformación de custumer

In [17]:
df_custumer['cus_Country']= df_custumer['cus_Country'].replace('UK','United Kingdom')
df_custumer['cus_Country']= df_custumer['cus_Country'].str.upper()
df_custumer['cus_Address']= df_custumer['cus_Address'].str.upper()
df_custumer['cus_City']= df_custumer['cus_City'].str.upper()

df_custumer.head(10)

,Id_custumer,cus_Address,cus_City,cus_Country,cus_Phone
0,ALFKI,OBERE STR. 57,BERLIN,GERMANY,030-0074321
1,ANATR,AVDA. DE LA CONSTITUCIÓN 2222,MÉXICO D.F.,MEXICO,(5) 555-4729
2,ANTON,MATADEROS 2312,MÉXICO D.F.,MEXICO,(5) 555-3932
3,AROUT,120 HANOVER SQ.,LONDON,UNITED KINGDOM,(171) 555-7788
4,BERGS,BERGUVSVÄGEN 8,LULEÅ,SWEDEN,0921-12 34 65
5,BLAUS,FORSTERSTR. 57,MANNHEIM,GERMANY,0621-08460
6,BLONP,"24, PLACE KLÉBER",STRASBOURG,FRANCE,88.60.15.31
7,BOLID,"C/ ARAQUIL, 67",MADRID,SPAIN,(91) 555 22 82
8,BONAP,"12, RUE DES BOUCHERS",MARSEILLE,FRANCE,91.24.45.40
9,BOTTM,23 TSAWASSEN BLVD.,TSAWASSEN,CANADA,(604) 555-4729


Transformación de employee

In [18]:
df_employees['emp_Country']= df_employees['emp_Country'].replace('UK','United Kingdom')
df_employees['emp_Country']= df_employees['emp_Country'].str.upper()
df_employees['emp_LastName']= df_employees['emp_LastName'].str.upper()
df_employees['emp_FirstName']= df_employees['emp_FirstName'].str.upper()
df_employees['emp_Title']= df_employees['emp_Title'].str.upper()
df_employees['emp_Address']= df_employees['emp_Address'].str.upper()
df_employees['emp_City']= df_employees['emp_City'].str.upper()

df_employees.head(10)

,emp_LastName,emp_FirstName,emp_Title,emp_Address,emp_City,emp_Country
0,DAVOLIO,NANCY,SALES REPRESENTATIVE,507 - 20TH AVE. E. APT. 2A,SEATTLE,USA
1,FULLER,ANDREW,"VICE PRESIDENT, SALES",908 W. CAPITAL WAY,TACOMA,USA
2,LEVERLING,JANET,SALES REPRESENTATIVE,722 MOSS BAY BLVD.,KIRKLAND,USA
3,PEACOCK,MARGARET,SALES REPRESENTATIVE,4110 OLD REDMOND RD.,REDMOND,USA
4,BUCHANAN,STEVEN,SALES MANAGER,14 GARRETT HILL,LONDON,UNITED KINGDOM
5,SUYAMA,MICHAEL,SALES REPRESENTATIVE,COVENTRY HOUSE MINER RD.,LONDON,UNITED KINGDOM
6,KING,ROBERT,SALES REPRESENTATIVE,EDGEHAM HOLLOW WINCHESTER WAY,LONDON,UNITED KINGDOM
7,CALLAHAN,LAURA,INSIDE SALES COORDINATOR,4726 - 11TH AVE. N.E.,SEATTLE,USA
8,DODSWORTH,ANNE,SALES REPRESENTATIVE,7 HOUNDSTOOTH RD.,LONDON,UNITED KINGDOM


Transformación de product

In [19]:
df_product['CategoryName']= df_product['CategoryName'].str.upper()
df_product['ProductName']= df_product['ProductName'].str.upper()
df_product.head(10)

,ProductName,CategoryName
0,CHAI,BEVERAGES
1,CHANG,BEVERAGES
2,ANISEED SYRUP,CONDIMENTS
3,CHEF ANTON'S CAJUN SEASONING,CONDIMENTS
4,CHEF ANTON'S GUMBO MIX,CONDIMENTS
5,GRANDMA'S BOYSENBERRY SPREAD,CONDIMENTS
6,UNCLE BOB'S ORGANIC DRIED PEARS,PRODUCE
7,NORTHWOODS CRANBERRY SAUCE,CONDIMENTS
8,MISHI KOBE NIKU,MEAT/POULTRY
9,IKURA,SEAFOOD


Transformación de Supplier

In [20]:
df_supplier['sup_Country']= df_supplier['sup_Country'].replace('UK','United Kingdom')
df_supplier['sup_CompanyName']= df_supplier['sup_CompanyName'].str.upper()
df_supplier['sup_Address']= df_supplier['sup_Address'].str.upper()
df_supplier['sup_City']= df_supplier['sup_City'].str.upper()
df_supplier['sup_Country']= df_supplier['sup_Country'].str.upper()
df_supplier.head(10)

,sup_CompanyName,sup_Address,sup_City,sup_Country
0,EXOTIC LIQUIDS,49 GILBERT ST.,LONDON,UNITED KINGDOM
1,NEW ORLEANS CAJUN DELIGHTS,P.O. BOX 78934,NEW ORLEANS,USA
2,GRANDMA KELLY'S HOMESTEAD,707 OXFORD RD.,ANN ARBOR,USA
3,TOKYO TRADERS,9-8 SEKIMAI MUSASHINO-SHI,TOKYO,JAPAN
4,COOPERATIVA DE QUESOS 'LAS CABRAS',CALLE DEL ROSAL 4,OVIEDO,SPAIN
5,MAYUMI'S,92 SETSUKO CHUO-KU,OSAKA,JAPAN
6,"PAVLOVA, LTD.",74 ROSE ST. MOONIE PONDS,MELBOURNE,AUSTRALIA
7,"SPECIALTY BISCUITS, LTD.",29 KING'S WAY,MANCHESTER,UNITED KINGDOM
8,PB KNÄCKEBRÖD AB,KALOADAGATAN 13,GÖTEBORG,SWEDEN
9,REFRESCOS AMERICANAS LTDA,AV. DAS AMERICANAS 12.890,SAO PAULO,BRAZIL


Transformación de Time

In [21]:
df_date['Name_Day']= df_date['Name_Day'].str.upper()
df_date['Name_Month']= df_date['Name_Month'].str.upper()
df_date.head(10)


,Date,Day,Month,Year,Quarter,Week,Weekday,Name_Day,Name_Month
0,2006-02-19,19,2,2006,1,7,6,SUNDAY,FEBRUARY
1,2006-02-20,20,2,2006,1,8,0,MONDAY,FEBRUARY
2,2006-02-21,21,2,2006,1,8,1,TUESDAY,FEBRUARY
3,2006-02-22,22,2,2006,1,8,2,WEDNESDAY,FEBRUARY
4,2006-02-23,23,2,2006,1,8,3,THURSDAY,FEBRUARY
5,2006-02-24,24,2,2006,1,8,4,FRIDAY,FEBRUARY
6,2006-02-25,25,2,2006,1,8,5,SATURDAY,FEBRUARY
7,2006-02-26,26,2,2006,1,8,6,SUNDAY,FEBRUARY
8,2006-02-27,27,2,2006,1,9,0,MONDAY,FEBRUARY
9,2006-02-28,28,2,2006,1,9,1,TUESDAY,FEBRUARY


Transformación de Hecho

In [22]:
df_hechos['Total']= round(df_hechos['Total'],2)
df_hechos.head(10)

,Id_custumer,Id_employee,Id_time,Id_supplier,Id_produc,Quantity,UnitPrice,Total
0,VINET,5,10248,5,11,12,14.0,168.0
1,VINET,5,10248,20,42,10,9.8,98.0
2,VINET,5,10248,14,72,5,34.8,174.0
3,TOMSP,6,10249,6,14,9,18.6,167.4
4,TOMSP,6,10249,24,51,40,42.4,1696.0
5,HANAR,4,10250,19,41,10,7.7,77.0
6,HANAR,4,10250,24,51,35,42.4,1484.0
7,HANAR,4,10250,2,65,15,16.8,252.0
8,VICTE,3,10251,9,22,6,16.8,100.8
9,VICTE,3,10251,26,57,15,15.6,234.0


#**CARGA DE DATOS AL DATA MART**

## **Conexión a la base de datos**


Conexión a la base de datos DW_estrella

In [23]:
path2 = "sqlite:///DW_estrella.db"
motor = sqlalchemy.create_engine(path2)
conectar = motor.connect()

FUNCION DE LA CARGA DE DATOS

In [24]:
def cargar_datos_a_sql(datos, connectar, tabla_Nombre):

    datos.to_sql(tabla_Nombre, conectar, if_exists='append', index=False)
    termino = print("Carga Finalizada")
    return termino

Carga de datos customer

In [25]:
tabla_nombre = "dim_Customer"
cargar_datos_a_sql(df_custumer, conectar, tabla_nombre)

Carga Finalizada


Carga de datos employees

In [26]:
tabla_nombre = "dim_Employee"
cargar_datos_a_sql(df_employees, conectar, tabla_nombre)

Carga Finalizada


Carga de datos Product

In [27]:
tabla_nombre = "dim_Product"
cargar_datos_a_sql(df_product, conectar, tabla_nombre)

Carga Finalizada


Carga de datos Supplier

In [28]:
tabla_nombre = "dim_Supplier"
cargar_datos_a_sql(df_supplier, conectar, tabla_nombre)

Carga Finalizada


Carga de datos Time

In [29]:
tabla_nombre = "dim_time"
cargar_datos_a_sql(df_date, conectar, tabla_nombre)

Carga Finalizada


Carga de datos a la tabla de hecho

In [30]:
tabla_nombre = "Fact_sales_order"
cargar_datos_a_sql(df_hechos, conectar, tabla_nombre)
conectar.close()


Carga Finalizada


# **MODELO COPO DE NIEVE**

**CREACIÓN DE LAS DIMENSIONES**

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 

CREAR DIMENSION 